# Notebook demo of using vqe module to solve a QUBO

In [1]:
from aquapointer.digital.loaddata import LoadData
from aquapointer.digital.qubo import Qubo
import aquapointer.digital.qubo_utils as qutils
from aquapointer.digital.vqe import VQE

# General imports
import numpy as np

# Pre-defined ansatz circuit, operator class
from qiskit.circuit.library import QAOAAnsatz
from qiskit.primitives import Sampler
from qiskit import transpile 
from qiskit import Aer

backend = Aer.get_backend('aer_simulator')
sampler = Sampler(options={"shots": int(1e4)})

In [2]:
# LoadData does all the file loading
ld = LoadData()

# Qubo computes all the qubo matrices given the 3d rism files and rescaled positions of registers from LoadData.
q = Qubo(ld)

In [3]:
# use the first slice as an example to solve
qubo, ising_ham = q.qubo_hamiltonian_pairs[1]
num_qubits = len(qubo)

# QAOA ansatz circuit
qaoa_ansatz = QAOAAnsatz(ising_ham, reps=1)

from qiskit.circuit.library import TwoLocal
# TwoLocal ansatz circuit
twolocal_ansatz = TwoLocal(num_qubits, 'ry', 'cx',  entanglement='linear', reps=1)

In [4]:
# classical brute-force solution
sol, ref_value = q.find_optimum(qubo=qubo)
sol, ref_value

('010000001', -0.05676551209085727)

In [10]:
def prob_optimal_solution(vqe_object: VQE, alpha: float, fraction: float, verbose=False) -> tuple[float, int]:

    res = vqe_object.run(alpha=alpha, method='COBYLA')
    #res = vqe_object.run_sharpe(method='COBYLA')
    nfev = res.nfev

    # Assign solution parameters to ansatz
    qc = vqe_object.ansatz.assign_parameters(vqe_object.params)
    # Add measurements to our circuit
    qc.measure_all()
    # Sample ansatz at optimal parameters
    samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
    samp_dist_binary = samp_dist.binary_probabilities()

    correct_dist = {}
    for key in samp_dist_binary.keys():
        reverse_key = key[::-1]
        keynot = [(int(b)+1)%2 for b in reverse_key]
        correct_dist[''.join(map(str, keynot))] = samp_dist_binary[key]

    prob_energy = []
    bitstrings = []
    for key in correct_dist.keys():
        key_np = np.fromiter(map(int, key), dtype=int)
        prob_energy.append([correct_dist[key], qutils.ising_energy(key_np, qubo)])
        bitstrings.append(key)

    bitstrings = np.array(bitstrings)
    prob_energy = np.array(prob_energy)

    sorted_indices = np.argsort(prob_energy[:, 1])
    sorted_keys = bitstrings[sorted_indices]
    sorted_values = prob_energy[:, 1][sorted_indices]

    opt_energy = sorted_values[0]
    opt_b = sorted_keys[0]
    
    #pick top 10% of lowest observed energies and compute probability mass on them
    n = int(len(prob_energy)*fraction)
    if verbose:
        if n<=1:
            print(prob_energy)

    total_mass = 0.0
    top_avg_energy = 0.0
    for i in range(n):
        total_mass += correct_dist[sorted_keys[i]]
        top_avg_energy += sorted_values[i]*correct_dist[sorted_keys[i]]
        if verbose:
            print(sorted_values[i], correct_dist[sorted_keys[i]])
    top_avg_energy = top_avg_energy / total_mass
    return round(top_avg_energy, 5), round(total_mass, 3), nfev, (opt_b, opt_energy)  

In [6]:
# run the optimization for a given vqe_object and a list of confidence intervals
def run_optimization(vqe_object, alphas):
    nfevs = [] #this is for book keeping the number of function evaluations    
    opt_energy = np.inf

    for alpha in alphas:
        top_avg_energy, total_mass, nfev, opt = prob_optimal_solution(vqe_object, alpha, 0.2, verbose=False)
        nfevs.append(nfev)
        if opt_energy>opt[1]:
            opt_energy = opt[1]
            print(opt)
        print(f"{alpha}, {top_avg_energy}, {total_mass}")
    return nfevs

In [12]:
alphas = [1.0, 0.75, 0.5, 0.25, 0.1]
#using QAOA ansatz
beta  = [0.7977]#, 0.7905, 0.5657]#, 0.4189]#, 0.3575, 0.3279, 0.2785, 0.1911, 0.1384, 0.0885]
gamma = [0.0765]#, 0.1634, 0.3662]#, 0.5890]#, 0.7046, 0.7594, 0.8345, 0.9352, 0.9529, 0.9976]
params = beta+gamma
vqe_qaoa = VQE(qubo=qubo, ansatz=qaoa_ansatz, ising_ham=ising_ham, sampler=sampler, params=params)

nfevs_qaoa = run_optimization(vqe_qaoa, alphas)

('010000001', -0.05676551209085727)
1.0, 0.03583, 0.068
0.75, -0.01518, 0.844
0.5, -0.01898, 0.896
0.25, -0.01953, 0.886
0.1, -0.01255, 0.687


In [13]:
#using linear entanglement ansatz
vqe_linear = VQE(qubo=qubo, ansatz=twolocal_ansatz, ising_ham=ising_ham, sampler=sampler, params=None)
alphas = [1.0, 0.75, 0.5, 0.25, 0.1]

nfevs_linear = run_optimization(vqe_linear, alphas)

('010000001', -0.05676551209085727)
1.0, -0.01761, 0.308
0.75, -0.04173, 0.832
0.5, -0.03979, 0.608
0.25, -0.01768, 0.484
0.1, -0.02245, 0.395


# Resource estimation

In [15]:
resources_linear = {}
resources_qaoa = {}

basis_gates=['u1', 'u2', 'u3', 'cx']

qaoa_ansatz_transpiled = transpile(qaoa_ansatz, backend, basis_gates=['u1', 'u2', 'u3', 'cx'], optimization_level=2)
gates_qaoa = qaoa_ansatz_transpiled.count_ops()

twolocal_ansatz_transpiled = transpile(twolocal_ansatz, backend, basis_gates=['u1', 'u2', 'u3', 'cx'], optimization_level=2)
gates_twolocal = twolocal_ansatz_transpiled.count_ops()

In [16]:
nfevs_linear

[170, 242, 147, 186, 149]

In [17]:
nfevs_qaoa

[29, 42, 30, 31, 30]

In [23]:
for gate in basis_gates:
    resources_linear[gate] = nfevs_linear[0] * gates_twolocal.get(gate, 0)
    resources_qaoa[gate] = nfevs_qaoa[0] * gates_qaoa.get(gate, 0)

In [24]:
resources_linear

{'u1': 0, 'u2': 0, 'u3': 3060, 'cx': 1360}

In [25]:
resources_qaoa

{'u1': 2349, 'u2': 261, 'u3': 261, 'cx': 4176}